# Carregar dados do Lakehouse Silver para Gold


# Transformações de Silver para Gold

A transformação para a camada Gold envolve:

**Agregações:** Realizar agregações como somas, médias, contagens, etc., para gerar insights de alto nível.

**Cálculos de Negócio:** Aplicar regras de negócios que extraem métricas e KPIs importantes para a análise.

**Desnormalização:** Pode ser necessário juntar ou combinar dados de diferentes fontes para facilitar a análise.

**Criação de Modelos Analíticos:** Aplicação de métricas de negócios complexas, como cálculos de tendências, segmentação de clientes, etc.

## Transformações: Nesse exemplo, estamos agregando os dados para calcular:

O total de quantidade vendida (SUM).
O preço médio unitário (AVG).
O número total de clientes distintos (COUNT DISTINCT).
A última data de venda (MAX).
Criação da Tabela Gold: A tabela Gold será criada na camada Gold e armazenará esses dados agregados e transformados.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Inicializando o SparkSession (se ainda não estiver ativo)
spark = SparkSession.builder.getOrCreate()

# Carregando a tabela Silver
df_silver = spark.table("Silver_Lakehouse_GL.produto")

# Realizando a transformação para a camada Gold
df_gold = (
    df_silver
    .groupBy("IDPRODUTO", "NOME")
    .agg(
        F.sum("VALOR_UNITARIO").alias("valor_total_vendido"),       # Soma dos valores unitários
        F.avg("CUSTO_MEDIO").alias("custo_medio_produto"),          # Média do custo médio
        F.countDistinct("ID_CATEGORIA").alias("total_categorias"),  # Contagem de categorias distintas
        F.max("MARGEM_BRUTA").alias("maior_margem_bruta")           # Maior margem bruta
    )
)

# Criar o esquema Gold_Lakehouse_GL se não existir
spark.sql("CREATE SCHEMA IF NOT EXISTS Gold_Lakehouse_GL")

# Alterar para o esquema correto
spark.sql("USE Gold_Lakehouse_GL")

# Salvando o resultado na camada Gold
df_gold.write.format("delta").mode("overwrite").saveAsTable("Gold_Lakehouse_GL.produto_gold")


# Validação


In [ ]:
# Listar todas as tabelas no catálogo atual
spark.sql("SHOW TABLES").show()

# Confirmar como a tabela está sendo salva
df_gold.write.format("delta").mode("overwrite").saveAsTable("produto_gold")

df_gold.write.format("delta").mode("overwrite").saveAsTable("Gold_Lakehouse_GL.produto_gold")

# Verificar o esquema atual
spark.sql("SELECT current_database()").show()

# Alterar para o esquema correto, se necessário
spark.sql("USE Gold_Lakehouse_GL")

spark.sql("SELECT * FROM produto_gold").show(5)

# Reexecutar transformação e salvar
df_gold.write.format("delta").mode("overwrite").saveAsTable("Gold_Lakehouse_GL.produto_gold")



## Melhorias na Tabela Gold
É possivel realizar outras melhorias para garantir que a camada Gold contenha dados prontos para consumo, como:

Limpeza de dados (remoção de registros nulos ou inconsistentes).
Transformações de tipo de dados.
Criação de índices ou outras otimizações.

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# Carregar a tabela Delta
delta_table = DeltaTable.forName(spark, "Gold_Lakehouse_GL.produto_gold")

# Deletar as linhas onde 'IDPRODUTO' é NULL
delta_table.delete(condition = F.col("IDPRODUTO").isNull())

# Performance

## Particionamento de Dados

Particionar sua tabela pode melhorar significativamente a performance de leitura, especialmente quando você tem grandes volumes de dados. O particionamento divide os dados em subpastas com base em uma ou mais colunas. Por exemplo, se você tem dados por ano ou mês, pode particionar os dados com base na coluna de data.

In [ ]:
## Exemplo: df_gold.write.partitionBy("ano_venda").format("delta").mode("overwrite").saveAsTable("Gold_Lakehouse_GL.produto_gold")

Neste cenario não é valido, pois não tem colunas adequadas

In [ ]:
Continuar a perfomance